In [1]:
import os
import utils

utils.load_env()
os.environ['LANGCHAIN_TRACING_V2'] = "true"

In [2]:
from langchain_core.messages import HumanMessage

# for llm model
from langchain_openai import ChatOpenAI
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from tools import find_place_from_text, nearby_search
from typing import Dict, List, Tuple
from langchain.agents import (
    AgentExecutor,
)
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Bind the tools to the model
tools = [find_place_from_text, nearby_search]  # Include both tools if needed

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

meta = utils.load_agent_meta()[0]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", meta['prompt']),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)


class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_types(
    input_type=AgentInput
)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [3]:
def submitUserMessage(user_input: str) -> str:
    responce = agent_executor.invoke({"input": user_input, "chat_history": []})
    return responce["output"]

submitUserMessage("ค้นหาร้านกาแฟใกล้มาบุญครอง")

Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')




> Entering new AgentExecutor chain...


Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')



Invoking: `nearby_search` with `{'input_dict': {'keyword': 'ร้านกาแฟ', 'location_name': 'มาบุญครอง', 'radius': 1000, 'place_type': 'cafe'}}`




Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')


number of results more than 20

        address: อาคารหอศิลปวัฒนธรรมแห่งกรุงเทพมหานคร(BACC) 939 Rama I Rd, Wang Mai, Pathum Wan

        location: {'lat': 13.7466896, 'lng': 100.5304572}

        name: Gallery Drip Coffee

        opening hours: {'open_now': True}

        rating: 4.6

        plus code: 7P52PGWJ+M5


        
        address: 445 Rama I Rd, Rong Muang, Pathum Wan

        location: {'lat': 13.7476317, 'lng': 100.5232939}

        name: D Coffee House

        opening hours: {'open_now': True}

        rating: 4.9

        plus code: 7P52PGXF+38


        
        address: 43/4 Rama VI Rd, Rong Muang, Pathum Wan

        location: {'lat': 13.7497223, 'lng': 100.521221}

        name: Olympic Coffee

        opening hours: {'open_now': True}

        rating: 4.8

        plus code: 7P52PGXC+VF


        
        address: 36, 6 Kasem San 1 Alley, Wang Mai, Pathum Wan

        location: {'lat': 13.7477989, 'lng': 100.5296324}

        name: Sip 'n Drip

        opening ho